In [1]:
using Pkg
Pkg.activate("..")


  Activating project at `/lustre/eaglefs/projects/pvb/cju/ProgressiveHedging.jl`


In [2]:
# ] add PowerSystemCaseBuilder

In [3]:
using Distributed
const WORKERS = 1 # Change to > 1 to use parallel
if nworkers() < WORKERS
    diff = (nprocs() == nworkers() ? WORKERS : WORKERS - nworkers())
    println("Adding $diff worker processes.")
    Distributed.addprocs(diff)
    # Make sure these workers also have an environment with PH installed
    @everywhere using Pkg
    for w in workers()
        @spawnat(w, Pkg.activate(".."))
    end
end

@everywhere using ProgressiveHedging
@everywhere const PH = ProgressiveHedging
@everywhere using Ipopt
@everywhere using JuMP, Xpress, PowerSimulations, PowerSystems, PowerSystemCaseBuilder, TimeSeries
@everywhere const PSI = PowerSimulations
@everywhere const PSY = PowerSystems
@everywhere const PSB = PowerSystemCaseBuilder


┌ Info: Xpress: Found license file /nopt/nrel/apps/xpressmp/8.13.0/bin/xpauth.xpr
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:44
┌ Info: Xpress: Development license detected.
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:89


In [4]:
using JuMP, Xpress, PowerSimulations, PowerSystems, PowerSystemCaseBuilder, InfrastructureSystems, Dates, TimeSeries
const PSI = PowerSimulations
const PSY = PowerSystems
const PSB = PowerSystemCaseBuilder
solver = optimizer_with_attributes(Xpress.Optimizer, "MIPRELSTOP" => 1e-4)

MathOptInterface.OptimizerWithAttributes(Xpress.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("MIPRELSTOP") => 0.0001])

In [5]:
system = PSB.build_system(SIIPExampleSystems, "5_bus_hydro_uc_sys_with_targets"; force_build =true, has_reserve=false);
for serv in get_components(Service, system)
    remove_component!(system, serv)
end
remove_time_series!(system, DeterministicSingleTimeSeries)
remove_time_series!(system, Deterministic)
for load in get_components(PowerLoad, system)
    ts = get_time_series(SingleTimeSeries, load, "max_active_power");
    ts_val = get_time_series_values(SingleTimeSeries, load, "max_active_power")
    println("(Number of datasets) >> ", size(ts_val))
    time_stamps = get_initial_timestamp(ts):Hour(1):get_initial_timestamp(ts)+Hour(335)
    remove_time_series!(system, SingleTimeSeries, load, "max_active_power")
    data = TimeArray(time_stamps, vcat(ts_val, ts_val + rand(168)*0.2))
    forecast = SingleTimeSeries("max_active_power", data;)
    add_time_series!(system, load, forecast)
end

for hyd in get_components(HydroDispatch, system), label in ["max_active_power"]
    ts = get_time_series(SingleTimeSeries, hyd, label);
    ts_val = get_time_series_values(SingleTimeSeries, hyd, label)
    time_stamps = get_initial_timestamp(ts):Hour(1):get_initial_timestamp(ts)+Hour(335)
    remove_time_series!(system, SingleTimeSeries, hyd, label)
    data = TimeArray(time_stamps, vcat(ts_val, ts_val + rand(168)*0.01))
    forecast = SingleTimeSeries(label, data;)
    add_time_series!(system, hyd, forecast)
end

labels = ["max_active_power", "inflow"]
for hyd in get_components(HydroEnergyReservoir, system), label in labels
    ts = get_time_series(SingleTimeSeries, hyd, label);
    ts_val = get_time_series_values(SingleTimeSeries, hyd, label)
    time_stamps = get_initial_timestamp(ts):Hour(1):get_initial_timestamp(ts)+Hour(335)
    @show time_stamps
    remove_time_series!(system, SingleTimeSeries, hyd, label)
    data = TimeArray(time_stamps, vcat(ts_val, ts_val + rand(168)*0.01))
    forecast = SingleTimeSeries(label, data;)
    add_time_series!(system, hyd, forecast)
end
PSY.transform_single_time_series!(system, 24, Hour(24))

new_cost= Dict(
    "Alta" => ThreePartCost((0.0, 14.0), 0.0, 4.0, 2.0),
    "Park City" => ThreePartCost((0.0, 15.0), 0.0, 1.5, 0.75),
    "Solitude" => ThreePartCost((0.0, 30.0), 0.0, 3.0, 1.5),
    "Sundance" => ThreePartCost((0.0, 40.0), 0.0, 4.0, 2.0),
    "Brighton" => ThreePartCost((0.0, 10.0), 0.0, 1.5, 0.75),
    )
for th in get_components(ThermalStandard, system)
    set_operation_cost!(th, new_cost[th.name])
end

# PSY.to_json(system, "5bus_hydro_2week.json"; force=true)

(Number of datasets) >> (168,)
(Number of datasets) >> (168,)
(Number of datasets) >> (168,)
time_stamps = DateTime("2020-01-01T00:00:00"):Hour(1):DateTime("2020-01-14T23:00:00")
time_stamps = DateTime("2020-01-01T00:00:00"):Hour(1):DateTime("2020-01-14T23:00:00")
time_stamps = DateTime("2020-01-01T00:00:00"):Hour(1):DateTime("2020-01-14T23:00:00")
time_stamps = DateTime("2020-01-01T00:00:00"):Hour(1):DateTime("2020-01-14T23:00:00")


┌ Info: Building new system 5_bus_hydro_uc_sys_with_targets from raw data
│   sys_descriptor.raw_data = /home/cju/.julia/packages/PowerSystemCaseBuilder/WBLmn/data
└ @ PowerSystemCaseBuilder /home/cju/.julia/packages/PowerSystemCaseBuilder/WBLmn/src/build_system.jl:38
┌ Info: Parsing csv data in Hydro_Upstream_Input.csv ...
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table_data.jl:144
┌ Info: Successfully parsed Hydro_Upstream_Input.csv
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table_data.jl:149
┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table_data.jl:144
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table_data.jl:149
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power

LoadError: InfrastructureSystems.ConflictingInputsError("forecast count 14 does not match system count 7")

In [6]:
sys_name = "/lustre/eaglefs/projects/pvb/5_bus_battery_testing/data/new_systems/pv=30_storagehours=10_wind=30/sys.json"
system = PSY.System(sys_name)

┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=/tmp/jl_8L2plJ compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(1, node_a, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/validation.jl:222
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(4, node_d, BusTypes.REF = 4, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}(

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ DEVICE_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 31          │
└───────────────────┴─────────────┘

Static Components
┌───────────────────┬───────┬────────────────────────┬───────────────┐
│ Type              │ Count │ Has Static Time Series │ Has Forecasts │
├───────────────────┼───────┼────────────────────────┼───────────────┤
│ Arc               │ 6     │ false                  │ false         │
│ Area              │ 1     │ false                  │ false         │
│ Bus               │ 5     │ false                  │ false         │
│ GenericBattery    │ 1     │ false                  │ false         │
│ Line              │ 6     │ false                  │ false         │
│ LoadZone          │ 1     │ false                  │ false         │
│ PowerLoad         │ 3     │ true                   │ false         │
│ RenewableDispatch │ 3     │ true                   │ false         │
│ ThermalStandard   │ 5     │ false                  │ false         │
└───────────────────┴───────┴────────────────────────┴───────────────┘

Time Series Summary
┌──────────────────────────────────┬────────────┐
│ Property                         │ Value      │
├──────────────────────────────────┼────────────┤
│ Components with time series data │ 6          │
│ Total StaticTimeSeries           │ 6          │
│ Total Forecasts                  │ 0          │
│ Resolution                       │ 60 minutes │
└──────────────────────────────────┴────────────┘

In [7]:
PSY.transform_single_time_series!(system, 168, Hour(168))

### Let's see what's under the hood

In [8]:
# set_interval!(system, 0)

In [9]:
# remove_time_series!(system, DeterministicSingleTimeSeries)

In [10]:
# system

In [11]:
# system.data.components.data

### Return to optimization problem setup

In [12]:
function build_operations_problem(
        system::PSY.System,
        initial_time,
        network = CopperPlatePowerModel,
        directory ="./simulation_folder/"
    )
    template_ed = ProblemTemplate(CopperPlatePowerModel)
    set_device_model!(template_ed, ThermalStandard, ThermalDispatchNoMin)
    set_device_model!(template_ed, PowerLoad, StaticPowerLoad)
    set_device_model!(template_ed, GenericBattery, BookKeeping)

    # set_device_model!(template_ed, HydroDispatch, FixedOutput)
    # set_device_model!(template_ed, HydroEnergyReservoir, HydroDispatchRunOfRiver)

    problem = DecisionModel(
        template_ed, 
        system, 
        name = "SubProblem",
        optimizer = solver, 
        # balance_slack_variables = true, # maybe this is depreciated
        warm_start = false,
        export_pwl_vars = true,
#         horizon = 2,
        initialize_model = false,
        initial_time = initial_time,
    )
    build!(problem, output_dir = directory)
    return problem
end
   
#= For some reason, the PSI.DecisionPr
function populate_ext!(problem::PSI.DecisionProblem, scenario_no, budget_data, total_scenarios)
    problem.ext["scenario_no"] = scenario_no
    problem.ext["budget_data"] = budget_data
    problem.ext["total_scenarios"] = total_scenarios
    return
end
=#
function populate_ext!(problem::DecisionModel{GenericOpProblem}, scenario_no, params)
    problem.ext["scenario_no"] = scenario_no
    problem.ext["efficiency_data"] = params["efficiency_data"]
    problem.ext["total_scenarios"] = params["total_scenarios"] 
    return
end

populate_ext! (generic function with 1 method)

We need to define the structs as not using the `PH.` classifier, e.g., `PH.PHFirstStage`, since they do not currently exist...

In [13]:
struct EnergyPH <: PSI.VariableType end

struct StorgeConsensusPH <: PSI.ConstraintType end

In [14]:
# function populate_PH_model!(problem::PSI.DecisionProblem)
function populate_PH_model!(problem::DecisionModel{GenericOpProblem})
    no_scenarios = problem.ext["total_scenarios"]
    time_no = problem.ext["scenario_no"]
    time_no += 1
    efficiency_data = problem.ext["efficiency_data"]
    # eff_data = problem.ext["eff_data"] # dictionary for each device
    
    optimization_container = PSI.get_optimization_container(problem)
    # https://github.com/NREL-SIIP/PowerSimulations.jl/search?q=time_steps
    # (Depreciated?) time_steps = PSI.model_time_steps(optimization_container)
    time_steps = PSI.get_time_steps(optimization_container)
    # println(">>> Time steps: ", time_steps)
    var_pin = PSI.get_variable(optimization_container, ActivePowerInVariable(), GenericBattery)
    var_pout= PSI.get_variable(optimization_container, ActivePowerOutVariable(), GenericBattery)
    var_e   = PSI.get_variable(optimization_container, EnergyVariable(), GenericBattery)
    e_balance = PSI.get_constraint(optimization_container, EnergyBalanceConstraint(), GenericBattery)
    
    # the first element is the name of the battery, the second is the number of batteries in...
    device_names, _ = axes(var_e)
    
    storage_consensus = PSI.add_constraints_container!(
        optimization_container,
        StorgeConsensusPH(),
        GenericBattery,
        device_names,
    )

    var_ph = PSI.add_variable_container!(
        optimization_container,
        EnergyPH(), 
        GenericBattery,
        device_names,
        1:no_scenarios,
    )
    jump_model = PSI.get_jump_model(optimization_container)
    for name in device_names
        for sec in 1:no_scenarios
            var_ph[name, sec] = JuMP.@variable(
                optimization_container.JuMPmodel,
                base_name = "Energy_{$(name)_{$(sec)}}",
            )
        end
        
        if time_no > 1
            (eff_in, eff_out) = efficiency_data[name]
            JuMP.delete(jump_model, e_balance[name, 1])
            
            e_balance[name, 1] = JuMP.@constraint(jump_model, 
                var_e[name, 1] == var_ph[name, time_no-1] + eff_in * var_pin[name,1] 
                                  - var_pout[name,1]/eff_out
            )
        end
        
        storage_consensus[name] = 
            JuMP.@constraint(jump_model, var_e[name, 24] == var_ph[name, time_no])
        
    end
    return
end   

populate_PH_model! (generic function with 1 method)

In [15]:
# function get_subproblem(problem::PSI.DecisionProblem)
function get_subproblem(problem::DecisionModel{GenericOpProblem})
    optimization_container = PSI.get_optimization_container(problem)
    jump_model = PSI.get_jump_model(optimization_container)
    
    first_stge_var =  JuMP.VariableRef[]
    secound_stge_var = JuMP.VariableRef[]
    # TODO: See what variables are here
    for (var_key, cont) in PSI.get_variables(problem) 
        # println("Var dict key val: ", var_key, " || ", cont)
        if var_key == PSI.VariableKey(EnergyPH, GenericBattery)
            push!(first_stge_var, cont...)
        else
            push!(secound_stge_var, cont...)
        end
    end
    variable_map = Dict{PH.StageID, Vector{JuMP.VariableRef}}(
        [
            PH.stid(1) => first_stge_var,
            PH.stid(2) => secound_stge_var,
        ])
    
    # This might be causing the slowdown
    # println("==================")
    # println(jump_model)
    # println("==================")
    return jump_model, variable_map
end
    

get_subproblem (generic function with 1 method)

In [16]:
function create_model(
    scenario_id::PH.ScenarioID,
    params;
    kwargs...
)
    # initial_time = [DateTime("2020-01-01T00:00:00"), DateTime("2020-01-08T00:00:00")]
    initial_time = DateTime("2024-01-01T00:00:00") + Dates.Day(PH.value(scenario_id))
    println(initial_time)
    system = PSY.System(sys_name)
    PSY.transform_single_time_series!(system, 24, Hour(24))
    problem = build_operations_problem(
        system,
        initial_time,
        CopperPlatePowerModel,
        "./simulation_folder/"
    )
    
    populate_ext!(problem, PH.value(scenario_id), params)
    populate_PH_model!(problem)
    jump_model, variable_map = get_subproblem(problem)
    return JuMPSubproblem(jump_model, scenario_id, variable_map)
end

create_model (generic function with 1 method)

In [17]:
function build_scen_tree(D)
    probs = [1.0/d for d=1:D]
    tree = PH.ScenarioTree()
    for l in 1:D
        PH.add_leaf(tree, tree.root, probs[l])
    end
    return tree
end

build_scen_tree (generic function with 1 method)

In [18]:
# Borrowed from previous notebook from Sourabh
efficiency_data = Dict()
for h in  get_components(GenericBattery, system)
    name = PSY.get_name(h)
    efficiency_data[name] = PSY.get_efficiency(h)
end
efficiency_data["Storage_10"]

(in = 0.85, out = 1.0)

In [19]:
params = Dict()
params["efficiency_data"] = efficiency_data

num_days = 14
params["total_scenarios"] = num_days

st = PH.two_stage_tree(num_days)

1;

In [22]:
ef_model = @time PH.solve_extensive(st,
    create_model, 
    ()->Xpress.Optimizer(),
    params,
)
# println(get_objective(ef_model))
objval = JuMP.objective_value(ef_model)
println("Obj val: ", objval)

┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=/tmp/jl_6IBsdj compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(1, node_a, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/validation.jl:222
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(4, node_d, BusTypes.REF = 4, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}(

2024-01-04T00:00:00
2024-01-09T00:00:00
2024-01-01T00:00:00
2024-01-05T00:00:00
2024-01-06T00:00:00
2024-01-12T00:00:00
2024-01-10T00:00:00
2024-01-08T00:00:00
2024-01-14T00:00:00
2024-01-02T00:00:00
2024-01-13T00:00:00
2024-01-07T00:00:00
2024-01-03T00:00:00
2024-01-11T00:00:00
  0.943665 seconds (2.68 M allocations: 184.242 MiB, 9.87% gc time, 2.08% compilation time)
Obj val: 113572.57142857143


┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=/tmp/jl_kxMaSq compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(1, node_a, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/validation.jl:222
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(4, node_d, BusTypes.REF = 4, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}(

In [21]:
@time (n, err, rerr, obj, soln, phd) = PH.solve(st,
                                          create_model,
                                          PH.ScalarPenaltyParameter(25.0), 
                                          params,
                                          atol=1e-8, rtol=1e-12, max_iter=100,
                                          report=10, # print residual info every 10 iterations
                                          )
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

Initializing...
...launching workers...
...initializing subproblems...


┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=/tmp/jl_mx7CT7 compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(1, node_a, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/validation.jl:222
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(4, node_d, BusTypes.REF = 4, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}(

2024-01-04T00:00:00
2024-01-09T00:00:00
2024-01-01T00:00:00
2024-01-05T00:00:00
2024-01-06T00:00:00
2024-01-12T00:00:00
2024-01-10T00:00:00
2024-01-08T00:00:00
2024-01-14T00:00:00
2024-01-02T00:00:00
2024-01-13T00:00:00
2024-01-07T00:00:00
2024-01-03T00:00:00
2024-01-11T00:00:00
Solving...
Iter:    0   AbsR: 5.665866e+00   RelR: 3.605551e+00   Xhat: 1.514266e+00   X: 5.459765e+00
Iter:   10   AbsR: 2.182931e-03   RelR: 2.441354e-04   Xhat: 1.668546e-03   X: 1.407530e-03
Iter:   20   AbsR: 1.669788e-03   RelR: 1.862745e-04   Xhat: 1.412185e-03   X: 8.910254e-04
Iter:   30   AbsR: 1.406208e-03   RelR: 1.566461e-04   Xhat: 9.751229e-04   X: 1.013191e-03
Iter:   40   AbsR: 1.178742e-03   RelR: 1.313082e-04   Xhat: 8.266307e-04   X: 8.403065e-04
Iter:   50   AbsR: 1.072482e-03   RelR: 1.194742e-04   Xhat: 7.396148e-04   X: 7.766518e-04
Iter:   60   AbsR: 9.851396e-04   RelR: 1.097475e-04   Xhat: 8.083052e-04   X: 5.631543e-04
Iter:   70   AbsR: 9.130171e-04   RelR: 1.017156e-04   Xhat: 5.85

┌ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=/tmp/jl_eW3kV0 compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/hdf5_time_series_storage.jl:100
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(1, node_a, BusTypes.PV = 3, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}())
└ @ InfrastructureSystems /home/cju/.julia/packages/InfrastructureSystems/Oc56m/src/validation.jl:222
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = magnitude
│   valid_range = voltage_limits
│   valid_info.ist_struct = Bus(4, node_d, BusTypes.REF = 4, 0.0, 0.01, (min = 0.95, max = 1.05), 100.0, Area(nothing, 0.0, 0.0, 0.0), LoadZone(zone1, 10.0, 3.287), Dict{String, Any}(